In [1]:
import autogen

config_list = autogen.config_list_from_dotenv(
    dotenv_file_path='../.env', # If None the function will try to find in the working directory
    filter_dict={
        "model": {
            "gpt-3.5-turbo"
        }
    }
)

In [2]:
# import random
# from typing import List, Dict

# class CustomGroupChat(GroupChat):
#     def __init__(self, agents, messages, max_round=10):
#         super().__init__(agents, messages, max_round)
#         self.previous_speaker = None  # Keep track of the previous speaker
    
#     def select_speaker(self, last_speaker: Agent, selector: AssistantAgent):
#         # Check if last message suggests a next speaker or termination
#         last_message = self.messages[-1] if self.messages else None
#         if last_message:
#             if 'NEXT:' in last_message['content']:
#                 suggested_next = last_message['content'].split('NEXT: ')[-1].strip()
#                 print(f'Extracted suggested_next = {suggested_next}')
#                 try:
#                     return self.agent_by_name(suggested_next)
#                 except ValueError:
#                     pass  # If agent name is not valid, continue with normal selection
#             elif 'TERMINATE' in last_message['content']:
#                 try:
#                     return self.agent_by_name('User_proxy')
#                 except ValueError:
#                     pass  # If 'User_proxy' is not a valid name, continue with normal selection
        
#         team_leader_names = [agent.name for agent in self.agents if agent.name.endswith('1')]

#         if last_speaker.name in team_leader_names:
#             team_letter = last_speaker.name[0]
#             possible_next_speakers = [
#                 agent for agent in self.agents if (agent.name.startswith(team_letter) or agent.name in team_leader_names) 
#                 and agent != last_speaker and agent != self.previous_speaker
#             ]
#         else:
#             team_letter = last_speaker.name[0]
#             possible_next_speakers = [
#                 agent for agent in self.agents if agent.name.startswith(team_letter) 
#                 and agent != last_speaker and agent != self.previous_speaker
#             ]

#         self.previous_speaker = last_speaker

#         if possible_next_speakers:
#             next_speaker = random.choice(possible_next_speakers)
#             return next_speaker
#         else:
#             return None

In [3]:
# Termination message detection
def is_termination_msg(content) -> bool:
    have_content = content.get("content", None) is not None
    if have_content and "TERMINATE" in content["content"]:
        return True
    return False

In [4]:
key_idea = """On planet Earth, life exists in hostile and extreme environments, and the organisms that survive there are termed extremophiles."""
key_idea_description = """Extremophiles are organisms that thrive in extreme conditions on Earth, such as high temperatures, acidity, alkalinity, or pressure. These environments include locations like deep-sea hydrothermal vents, acidic hot springs, or subglacial lakes in Antarctica. The adaptability of these organisms expands our understanding of life's limits and opens up possibilities for life in extreme conditions beyond Earth."""
theory = """Extremophiles and Their Environments
Definition: Extremophiles are primarily unicellular microbes, including bacteria and archaea, that live in harsh environments​​.
Habitats: These environments include deep-sea hydrothermal vents with high pressure and temperatures (up to 122 °C), volcanic hot springs, deep underground in mines, and in bodies of water that are highly acidic, alkaline, extremely salty, or even radioactive​​.
The Significance of Extremophiles
Adaptation and Survival: Extremophiles have adapted to survive in conditions that are generally detrimental to most life forms. Their existence expands our understanding of the limits and versatility of life.
Implications for Extraterrestrial Life: The study of extremophiles has significant implications for astrobiology. The discovery of diverse microbial communities in extreme environments like subglacial lakes under the Antarctic ice sheet suggests that life might exist in similar extreme conditions elsewhere in the universe, such as on ice-covered moons like Europa​​.
Europa: A Candidate for Extraterrestrial Life
Characteristics of Europa: Europa, a moon of Jupiter, is a primary candidate for extraterrestrial life exploration. It has an ice-covered surface, many kilometers thick, with a possible sub-surface ocean of salty water. The surface of Europa stretches and relaxes due to tidal movements caused by its elliptical orbit around Jupiter, generating heat that could keep a sub-surface ocean in liquid state​​.
Potential Habitability: Europa is considered a promising location for life beyond Earth due to its thin ice shell, liquid ocean, and contact with a geologically active rocky core. The moon has water and the right chemical elements, along with a potentially stable environment​​.
Broader Understanding
Cellular Requirements for Life: For life to exist, certain conditions are necessary, including the availability of an energy source, liquid water for biochemical reactions, and essential chemical building blocks like carbon, oxygen, nitrogen, and hydrogen. Life can thrive in stable environmental conditions within a specific range of pressure, temperature, light intensity, pH, and salinity​​.
Challenging Traditional Concepts: Extremophiles challenge the traditional concept of habitable environments, indicating that life can exist in conditions previously thought to be uninhabitable. This expands our understanding of possible habitats for life, both on Earth and potentially in extraterrestrial settings.
This comprehensive overview illustrates the remarkable adaptability of life and opens up new possibilities for understanding life's potential existence in extreme environments, both on Earth and beyond."""
student_year_level = "Year 11"
subject = "Biology"
question_difficulty = "moderate"
question_examples_easy = ["What is an extremophile? An extremophile is an organism that thrives in extreme environmental conditions that are typically detrimental to most life forms.", "Give one example of an extreme environment on Earth. One example is deep-sea hydrothermal vents.", "What is unique about Europa, one of Jupiter's moons? Europa has an ice-covered surface and is thought to have a sub-surface ocean of salty water."]
question_examples_moderate = ["How do extremophiles survive in high temperatures? Extremophiles in high temperatures have specialized proteins and enzymes that remain stable and functional at elevated temperatures.", "Why is the study of extremophiles important? Studying extremophiles helps understand the adaptability and limits of life, and it may provide insights into the possibility of life on other planets or moons.", "Why is Europa considered a likely place to find life beyond Earth? Europa is considered likely because of its ice shell, liquid ocean, and geologically active core, which provide conditions potentially suitable for life."]
question_examples_hard = ["Explain how extremophiles challenge the traditional concept of habitable environments. Extremophiles challenge the traditional concept by surviving in environments previously thought to be uninhabitable, expanding our understanding of where life can exist", "Discuss the potential implications of finding life in subglacial lakes in Antarctica for the search for extraterrestrial life. The discovery in subglacial lakes suggests that similar extreme, ice-covered environments in the solar system, such as on Europa, could also harbor life.", "How does the study of extremophiles contribute to our understanding of the potential for life on planets and moons outside the 'Goldilocks Zone'? The study of extremophiles indicates that life can exist in conditions not reliant on a star's warmth, suggesting that planets and moons outside the 'Goldilocks Zone' could also support life."]
subject = 'biology'
question_examples = question_examples_moderate
other_system_instructions = ""
student_interests = ["gaming", "politics", "philosophy"]
student_career_goals = ["lawyer", "doctor", "engineer"]

In [5]:
gpt_config = {
    "cache_seed": 42,
    "temperature": 0,
    "config_list":config_list,
    "timeout":120,
}

Human = autogen.UserProxyAgent(
    name = "Human_Student",
    system_message = "A human student who will answer the question generated by the teacher and will be marked by the grader. The student can ask the Question Clarifier for clarification on questions posed by the teacher. If the student has finished a question (or before one is asked) they can ask questions of the theory discusser instead",
    code_execution_config=False
)
question_generator = autogen.AssistantAgent(
    name = "AI_Teacher",
    llm_config=config_list,
    system_message=f"""You are a professional question writer for textbooks and exams. Your task is to craft a {subject} question suitable for an Australian highschool student in {student_year_level}. This question will be given to a human student to answer. Your question should be based on the following information:

- This is the "key idea" being assessed and what your question explore: {key_idea}.
- Ensure the question aligns with these complexities and nuances: {key_idea_description}.
- Here is some additional theory surrounding the key idea. This information is relevant to helping you formulate the question but you do not have to strictly follow it. Take some personal liberty while still being within the same context governed by the key idea: {theory}.
- Student Year Level - It is important you create create a question that is appropriate for this a student at this academic level: {student_year_level}.
- Ensure the question pertains to and enriches understanding in this field: {subject}.
- Difficulty Level - The question's complexity should be tailored to this setting: {question_difficulty}.

Where applicable, embed the student's interests or career goals into the question to enhance engagement. If these elements don't directly align with the academic content, focus on the educational aspect.

- Student Interests - Consider these to make the question more engaging: {student_interests}.
- Student Career Goals - Incorporate these where relevant: {student_career_goals}.

Directly pose the question without prefacing it as a 'question' or any thing else like that. The question should be formatted as if it were written in a textbook or exam, ensuring it adheres to the specified difficulty level and educational goals.

Formulate a question that is academically suitable for a student at the {student_year_level} level. The question should match the specified difficulty of '{question_difficulty}'. Here are some example questions which are about the same key idea and at a similar difficulty. Do not questions directly, they are just meant to be examples of the expected difficulty:

Example Questions: {question_examples}

Additional Instructions: {other_system_instructions}"""
)

AI_student = autogen.AssistantAgent(
    name = "AI_student",
    llm_config=config_list,
    system_message= f"""You are an AI Australian {student_year_level} {subject} student. Your task is to answer the question generated by the AI Teacher. Your answer should be at the sophisitcation which is appropriate to an Australian student such as yourself, answering to a depth that would be exepected from a student in {student_year_level} and nothing more. Your answer should be base on the following information:
- Key Idea: {key_idea}. This is the primary concept that the question is exploring and the one you should focus on.
- Detailed Description of Key Idea: {key_idea_description}. Use this information to ensure that your answer fully encompasses the nuances of the key idea.
- Relevant Theory: {theory}. This is additional information surrounding the 'key idea'. It may or may not necessarily be directly related, use your own discretion.
- Student Year Level: {student_year_level}. Tailor the sophistication and depth of your answer to be appropriate for a student at this academic level.
- Additional Instructions: {other_system_instructions}. Consider these instructions to guide the style, format, or additional content of your answer.

Your response should directly address the question, formulated in a clear and concise manner, suitable for the specified student year level. Avoid prefacing your response with 'Answer:' or similar qualifiers. Instead, provide a straightforward explanation or solution as one would respond in an exam or to a textbook question. Your answer will be the solution used to grade another student's response, so ensure it is accurate and complete. The Grader will use your answer as a benchmakr for the human student's response.

Consider the depth of your response, ensuring that it is appropriate for the student's year level. Your answer should be sufficiently detailed to provide a comprehensive explanation, but not so detailed that it that it would be unusual for a student at a {student_year_level} academic level to provide so much depth. Your answer should be accurate and relevant to the question, avoiding unnecessary or irrelevant information."""
)

AI_quality_control = autogen.AssistantAgent(
    name = "AI_Quality_Control",
    llm_config= config_list,
    system_message="""You are an expert AI who's job is to ensure ensure the information posed by the AI in this groupchat is accurate and up to standard with the prompts they have been given"""
)

AI_marker = autogen.AssistantAgent(
    name = "Grader",
    llm_config=config_list,
    system_message = f"""
You are an AI evaluator tasked with assessing an Australian high school student's answer to a given question, providing a grade, and offering feedback based on their performance and number of attempts. Your evaluation should focus on the accuracy and understanding of the key idea and relevant theory, appropriate to the student's academic level.

Your task involves the following steps:

1. **Analyzing the Student's Answer**:
   - Analyze the student's answer against the key idea, relevant theory, and solution provided.
   - Identify errors or misconceptions in the student's response.
   - Use theAI student's answer as a benchmark for what is expected of {student_year_level} student.

2. **Providing Feedback**:
   - Based on the score and the number of attempts, decide on the type of feedback to provide.
   - If the score is high (80-100) and the answer mostly correct, congratulate the student and point out any minor misconceptions or areas for improvement.
   - If the score is lower and this is not the third attempt, encourage the student to try again, providing guidance on what aspects need more focus.
   - If this is the third attempt or the student has significant misconceptions, explain the correct answer and the reasoning behind it, being considerate and supportive.

3. **Giving Grade**
    - AFTER you have analyzed the students answer and provided feedback, Assign a score out of 100, considering the student's academic level and the completeness of their understanding of the key concepts.

3. **Next Steps**:
   - Ask the student if they would like to move on to the next question, discuss additional theory about the current question, or clarify any misunderstandings about the question itself.

Construct a response that includes the score, feedback, and next steps, formatted appropriately.

Here is some context to guide your evaluation and feedback:

- Key Idea: {key_idea}
- Description of Key Idea: {key_idea_description}
- Relevant Theory: {theory}
- Student Year Level: {student_year_level}
- Additional Instructions: {other_system_instructions}
"""
)

discuss_theory = autogen.AssistantAgent(
    name = "Theory_Discusser",
    llm_config=config_list,
    system_message = f""" You are an AI teaching assistant who discusses the theory surrounding a question the student has just answered You now tasked with answering questions about the theory related to the question. Your answers should be appropriate for the student's academic level, avoiding the use of sophisticated jargon unless directly asked. If relevant, tailor your discussion of the theroy around the question that was provided to the student, the student's answer(s) and the feedback you have provided them.

At the end of every message you send to the student, ask them if they would like to move on to the next question or if they would like to keep discussing more theory. You are the theory discusser. When prompted by the human student you should discuss the theory the student wants to go over.You are only to answer the human student's questions if AND ONLY IF the student isn't currently answering a question. You are only to respond to the human student if they have already answered the question correctly or if the Grader has told decided that the human student they are incorrect and already provided the soltion. It is very important you do not reveal the answer to the student while they still ahven't answered the question.

Here is some relevant infromation reagrding the question the student was asked to answer: 

- Key Idea - Assess/Evaluate the student's understanding and expression of this concept: {key_idea}.
- Description of Key Idea - Use this information to ensure that your evaluation fully encompasses the nuances of the key idea: {key_idea_description}. 
- Relevant Theory - Look for key differences/Assess how well the student's answer reflects an understanding of this theory: {theory}.
- Student Year Level - Evaluate/Consider the academic level of the student, focusing on their grasp of the concept rather than the depth of the response: {student_year_level}.
- Additional Instructions  Consider these instructions to guide the style, format, or additional content of your answer: {other_system_instructions}.

""")

clarifying_AI = autogen.AssistantAgent(
    name = "Question_Clarifier",
    llm_config=config_list,
    system_message = f"""You are an AI assistant that has been helping An Australian high school student in {student_year_level} learn by asking questions in the field of {subject}. They are now asking to clarify something about a question you have just asked but which they have yet to answer correctly. Additional theory surrounding a key idea which will be provided, along side some theory about the topic which is appropriate for their academic level. You are now tasked with answering questions about the theory related to the question. Your answers should be appropriate for the student's academic level, avoiding the use of sophisticated jargon unless directly asked. If relevant, tailor your discussion of the theroy around the question that was provided to the student, the student's answer(s) and the feedback you have provided them
    
It is critical you do not reveal the any solutions as the student is still attempting to answer the question and may attempt to do so again. As an example, if the question you have asked them is "what does the mitochondria do" and the student says "what's the difference between the the mitochondria and the golgi apparatus" then you should try to avoid answering their clarification question in a way that might reveal what the mitochondria does, for example by saying "well the golgi apparatus does not generate most of the chemical energy needed to power the cell's biochemical reactions." as this would reveal the answer to the student. Do not strictly follow this format, it is just an example but it captures the essence of what you should do.

After you provide your clarification to the student, ask the student if they are ready to answer the question or if they need more clarification.""")

# initial_message = """
# Work together to create a question for a human student. The general flow of the conversation should be as follows:

# 1. Teacher makes a question based on the provided information.
# The question should be sent to the human student for them to answer.
# 2. While the human student is answering the question, AI student answers the question before the human student
# 3. AI quality control checks that the answer the AI student gave is correct and of good quality
# 4. wait for the Human student to provide an answer
# 5. Grader will take the human answer and grade it
# 6. Depending on the result of the grade, the Human student will either be congratulated or asked to try again if they got it wrong (less then 80%) or they the grader will tell the student they are incorrect and provide the solution if they got it wrong three times.
# 7. If the student wants to discuss theory with the theory discusser, they can do so.
# 8. If the students want to move on to the next question, then the teacher will generate a question and the cycle will repeat.
# If the student has not answered the question correctly yet, or if the student has not answered it incorrectly 3 times yet, then if they ask for clarification on the question they are asnwering then Question Clarifier will help clarify questions
# """

groupchat = autogen.GroupChat(agents = [Human, question_generator, AI_student, AI_quality_control, AI_marker, discuss_theory, clarifying_AI],messages=[], max_round=50)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gpt_config)

In [6]:
Human.initiate_chat(
    manager,
    message="""
Work together to create a question for a human student. The general flow of the conversation should be as follows:

1. Teacher makes a question based on the provided information.
The question should be sent to the human student for them to answer.
2. While the human student is answering the question, AI student answers the question before the human student
3. AI quality control checks that the answer the AI student gave is correct and of good quality
4. wait for the Human student to provide an answer
5. Grader will take the human answer and grade it
6. Depending on the result of the grade, the Human student will either be congratulated or asked to try again if they got it wrong (less then 80%) or they the grader will tell the student they are incorrect and provide the solution if they got it wrong three times.
7. If the student wants to discuss theory with the theory discusser, they can do so.
8. If the students want to move on to the next question, then the teacher will generate a question and the cycle will repeat.
If the student has not answered the question correctly yet, or if the student has not answered it incorrectly 3 times yet, then if they ask for clarification on the question they are asnwering then Question Clarifier will help clarify questions
If at any point the student says they don't know the answer then Question_Clarifer should step in to offer to clarify the question"
If the student says something unreleated to the question you have just asked them then the the agent who has most recently spoken to them should tespond to the student by saying that they are not answering the question and that they should try to answer the question instead.
""",
)

Human_Student (to chat_manager):


Work together to create a question for a human student. The general flow of the conversation should be as follows:

1. Teacher makes a question based on the provided information.
The question should be sent to the human student for them to answer.
2. While the human student is answering the question, AI student answers the question before the human student
3. AI quality control checks that the answer the AI student gave is correct and of good quality
4. wait for the Human student to provide an answer
5. Grader will take the human answer and grade it
6. Depending on the result of the grade, the Human student will either be congratulated or asked to try again if they got it wrong (less then 80%) or they the grader will tell the student they are incorrect and provide the solution if they got it wrong three times.
7. If the student wants to discuss theory with the theory discusser, they can do so.
8. If the students want to move on to the next question, t

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens. However, your messages resulted in 5049 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}